# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.111458e+02     1.602309e+00
 * time: 0.08865690231323242
     1     1.032370e+01     9.013000e-01
 * time: 0.9211039543151855
     2    -1.180597e+01     9.773257e-01
 * time: 1.0743300914764404
     3    -3.423391e+01     7.632150e-01
 * time: 1.2201869487762451
     4    -4.767034e+01     5.998520e-01
 * time: 1.3742458820343018
     5    -5.705835e+01     1.912968e-01
 * time: 1.5231659412384033
     6    -5.990912e+01     1.494964e-01
 * time: 1.6292619705200195
     7    -6.098549e+01     4.752778e-02
 * time: 1.7438631057739258
     8    -6.140449e+01     5.359633e-02
 * time: 1.8516108989715576
     9    -6.164689e+01     5.284728e-02
 * time: 1.9659419059753418
    10    -6.184497e+01     2.724058e-02
 * time: 2.0766921043395996
    11    -6.198460e+01     1.989916e-02
 * time: 2.1923530101776123
    12    -6.205310e+01     1.761583e-02
 * time: 2.299510955810547
    13    -6.211448e+01     1.546765e-02
 * time: 2.4142010211

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557734
    AtomicNonlocal      14.8522691
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666458768
